In [1]:
from GBP.data import DataGenerator
from GBP.gbp import run_GaBP_SYNC_ACCELERATED, run_GaBP_HARDWARE_ACCELERATED_RESIDUAL_T_nP, run_GaBP_HARDWARE_ACCELERATED_RESIDUAL_T_MAX, run_GaBP_HARDWARE_ACCELERATED, run_GaBP_HARDWARE_ACCELERATED_RESIDUAL, run_GaBP_HARDWARE_ACCELERATED_RESIDUAL_NEW, run_GaBP_HARDWARE_ACCELERATED_EXCLUSION_NEIGHBOURS_ACROSS_STREAMS, run_GaBP_HARDWARE_ACCELERATED_EXCLUSION_NEIGHBOURS_ACROSS_STREAMS_STOCHASTIC
# , run_GaBP_HARDWARE_ACCELERATED_EXCLUSION_NEIGHBOURS_ACROSS_2_STREAMS
from GBP.utilities import HiddenPrints
from GBP.visulisation import set_plot_options, get_plot_colors, NetworkxGraph, AnalyzeResult
 
import warnings
import matplotlib
import numpy as np
import math
import random

# Option 1: Suppress all warnings
warnings.filterwarnings("ignore")

set_plot_options()
colors = get_plot_colors()

data_gen = DataGenerator()
result_analyzer = AnalyzeResult()

# SETUP

In [26]:
# # Number of nodes
# num_nodes = 1000
# sync_convergence_threshold = 1*10**-8

# # starting mae for convergence
# starting_mae_for_simulation = 10

# # RUN FIRST TIME
# sum_of_iterations = 0
# num_iterations = 1
# for _ in range(1000):
#     A,b = data_gen.get_1D_line_matrix_PSD_difficult(num_nodes, eigenvalue_spread=1e-5, regularization_strength=1e-2, noise_strength=1e-2)
#     graph = NetworkxGraph(A)
#     P_i, mu_i, N_i, P_ii, mu_ii, P_ij, mu_ij, iter_dist, stand_divs, means, iteration = run_GaBP_SYNC_ACCELERATED(A, b, max_iter=1000, mae=False, convergence_threshold=sync_convergence_threshold, show=True)
#     sum_of_iterations += iteration
#     final_mean = list(mu_i)
#     final_std = P_i
#     if iteration < 1000:
#         break
#     else:
#         print("=========== RESTART ===========")

# print(f"AVE. SYNC ITERATIONS = {sum_of_iterations/num_iterations}")

In [27]:
# Number of nodes
num_nodes = 1000
sync_convergence_threshold = 1*10**-8

# starting mae for convergence
starting_mae_for_simulation = 10

# RUN FIRST TIME
sum_of_iterations = 0
num_iterations = 1
for _ in range(1000):
    A,b = data_gen.get_2D_lattice_matrix_PSD_difficult(int(math.sqrt(num_nodes)), int(math.sqrt(num_nodes)), eigenvalue_spread=1e-5, regularization_strength=1e-2, noise_strength=1e-2)
    graph = NetworkxGraph(A)
    P_i, mu_i, N_i, P_ii, mu_ii, P_ij, mu_ij, iter_dist, stand_divs, means, iteration = run_GaBP_SYNC_ACCELERATED(A, b, max_iter=1000, mae=False, convergence_threshold=sync_convergence_threshold, show=True)
    sum_of_iterations += iteration
    final_mean = list(mu_i)
    final_std = P_i
    if iteration < 1000:
        break
    else:
        print("=========== RESTART ===========")

print(f"AVE. SYNC ITERATIONS = {sum_of_iterations/num_iterations}")

iteration: 1
1.241636976720013
-----
iteration: 2
0.3168349568300044
-----
iteration: 3
0.09761634655890773
-----
iteration: 4
0.08231894128090327
-----
iteration: 5
0.01828924658247927
-----
iteration: 6
0.012871557418050912
-----
iteration: 7
0.004858606808492514
-----
iteration: 8
0.005825997938050852
-----
iteration: 9
0.002484611759084568
-----
iteration: 10
0.0020867960091114236
-----
iteration: 11
0.0010600033196335493
-----
iteration: 12
0.0007812124360505655
-----
iteration: 13
0.000411074029904483
-----
iteration: 14
0.0002961357855315575
-----
iteration: 15
0.0001578324508144957
-----
iteration: 16
0.0001128862395486366
-----
iteration: 17
6.013600239444695e-05
-----
iteration: 18
4.3174285624682065e-05
-----
iteration: 19
2.2839504177123312e-05
-----
iteration: 20
1.6522162671376846e-05
-----
iteration: 21
8.670068764023755e-06
-----
iteration: 22
6.319427686505854e-06
-----
iteration: 23
3.2937296955620357e-06
-----
iteration: 24
2.4145859254780203e-06
-----
iteration: 25


### 

In [28]:
async_convergence_threshold = 1*10**-5
sum_of_iterations = 0
num_iterations = 1
NODE_UPDT_PE = 1
PEs = 1
for it in range(0,num_iterations):
    # print(f"-------------- ITERATION = {it+1} --------------")
    P_i, mu_i, iteration, STARTING_MAE = run_GaBP_HARDWARE_ACCELERATED(A, b, caching=False, node_updates_per_pe=NODE_UPDT_PE, number_pes=PEs, TRUE_MEAN=final_mean, max_iter=1, mae=True, convergence_threshold=async_convergence_threshold, show=True)
    sum_of_iterations += iteration
    print(f"-------------- ITERATION = {it+1} => Streams = {iteration} --------------")

print(f"AVE. ASYNC ITERATIONS = {sum_of_iterations/num_iterations}")
print(STARTING_MAE)

iteration: 1
1.2014502422925661
-----
iteration: 2
1.201342230627726
-----
------------ EARLY EXIT ------------
-------------- ITERATION = 1 => Streams = 3 --------------
AVE. ASYNC ITERATIONS = 3.0
1.201450242292566


In [30]:
# Number of nodes
num_nodes = 1000
sync_convergence_threshold = 1*10**-8

# starting mae for convergence
starting_mae_for_simulation = 10

# RUN FIRST TIME
sum_of_iterations = 0
num_iterations = 1
for _ in range(1000):
    b = starting_mae_for_simulation * b / STARTING_MAE
    graph = NetworkxGraph(A)
    P_i, mu_i, N_i, P_ii, mu_ii, P_ij, mu_ij, iter_dist, stand_divs, means, iteration = run_GaBP_SYNC_ACCELERATED(A, b, max_iter=200, mae=False, convergence_threshold=sync_convergence_threshold, show=False)
    sum_of_iterations += iteration
    final_mean = list(mu_i)
    final_std = P_i
    if iteration < 100000:
        break
    else:
        print("=========== RESTART ===========")

print(f"AVE. SYNC ITERATIONS = {sum_of_iterations/num_iterations}")

AVE. SYNC ITERATIONS = 35.0


### START SIMULATION

In [31]:
async_convergence_threshold = 1*10**-5

sum_of_iterations = 0
num_iterations = 3


NODE_UPDT_PE = 137
PEs = 5

for it in range(0,num_iterations):
    # print(f"-------------- ITERATION = {it+1} --------------")
    iteration = 100
    while iteration > 100-1:
        P_i, mu_i, iteration, _ = run_GaBP_HARDWARE_ACCELERATED(A, b, caching=False, node_updates_per_pe=NODE_UPDT_PE, number_pes=PEs, TRUE_MEAN=final_mean, max_iter=100, mae=True, convergence_threshold=async_convergence_threshold, show=True)
    sum_of_iterations += iteration
    print(f"-------------- ITERATION = {it+1} => Streams = {iteration} --------------")

print(f"AVE. ASYNC ITERATIONS = {sum_of_iterations/num_iterations}")

iteration: 1
83.23274362921882
-----
iteration: 2
49.36002598619407
-----
iteration: 3
37.57279603057147
-----
iteration: 4
23.71522347815918
-----
iteration: 5
14.723515641104726
-----
iteration: 6
11.812396002665146
-----
iteration: 7
7.174701581586903
-----
iteration: 8
5.172000654859906
-----
iteration: 9
3.658936193207166
-----
iteration: 10
2.589453942613452
-----
iteration: 11
2.181809764505384
-----
iteration: 12
1.7632590589993569
-----
iteration: 13
1.3670381648260517
-----
iteration: 14
1.220452030874276
-----
iteration: 15
0.8090330834505529
-----
iteration: 16
0.6596036485257383
-----
iteration: 17
0.5574773735634909
-----
iteration: 18
0.36857237974719237
-----
iteration: 19
0.3029476902213252
-----
iteration: 20
0.2379581251639506
-----
iteration: 21
0.19003022393797073
-----
iteration: 22
0.13788677672697072
-----
iteration: 23
0.1177436269239959
-----
iteration: 24
0.08798881225253914
-----
iteration: 25
0.08617709567781703
-----
iteration: 26
0.07856469050226067
-----

In [34]:
async_convergence_threshold = 1*10**-5

sum_of_iterations = 0
num_iterations = 1


NODE_UPDT_PE = 137
PEs = 4

for it in range(0,num_iterations):
    # print(f"-------------- ITERATION = {it+1} --------------")
    iteration = 250
    while iteration > 250-1:
        P_i, mu_i, iteration, _, _ = run_GaBP_HARDWARE_ACCELERATED_RESIDUAL_T_MAX(A, b, ratio=1.0, caching=True, node_updates_per_pe=NODE_UPDT_PE, number_pes=PEs, TRUE_MEAN=final_mean, max_iter=250, mae=True, convergence_threshold=async_convergence_threshold, show=True)
        break
    sum_of_iterations += iteration
    print(f"-------------- ITERATION = {it+1} => Streams = {iteration} --------------")

print(f"AVE. ASYNC ITERATIONS = {sum_of_iterations/num_iterations}")

iteration: 1
6.435668615220235
-----
iteration: 2
4.942091904902665
-----
iteration: 3
4.40102043995324
-----
iteration: 4
3.367896737736807
-----
iteration: 5
2.813845459001973
-----
iteration: 6
2.112819103589861
-----
iteration: 7
1.8076862960056659
-----
iteration: 8
1.4805252790549477
-----
iteration: 9
1.3075469199654004
-----
iteration: 10
1.1260444212386536
-----
iteration: 11
1.0323082900046812
-----
iteration: 12
0.8665291768649777
-----
iteration: 13
0.7543085611907172
-----
iteration: 14
0.6544489837305867
-----
iteration: 15
0.5939266298770756
-----
iteration: 16
0.46382739483643404
-----
iteration: 17
0.3959938902529579
-----
iteration: 18
0.3560956343625914
-----
iteration: 19
0.28352983284253114
-----
iteration: 20
0.26572840797503317
-----
iteration: 21
0.22368629373849974
-----
iteration: 22
0.1854440697456901
-----
iteration: 23
0.16188964656346277
-----
iteration: 24
0.14556798802318707
-----
iteration: 25
0.12693286939510065
-----
iteration: 26
0.1110490478964194
-

In [35]:
async_convergence_threshold = 1*10**-5

sum_of_iterations = 0
num_iterations = 3


NODE_UPDT_PE = 137
PEs = 4

for it in range(0,num_iterations):
    # print(f"-------------- ITERATION = {it+1} --------------")
    iteration = 250
    while iteration > 250-1:
        P_i, mu_i, iteration = run_GaBP_HARDWARE_ACCELERATED_RESIDUAL(A, b, caching=True, node_updates_per_pe=NODE_UPDT_PE, number_pes=PEs, TRUE_MEAN=final_mean, max_iter=250, mae=True, convergence_threshold=async_convergence_threshold, show=True)
    sum_of_iterations += iteration
    print(f"-------------- ITERATION = {it+1} => Streams = {iteration} --------------")

print(f"AVE. ASYNC ITERATIONS = {sum_of_iterations/num_iterations}")

iteration: 1
6.435668615220235
-----
iteration: 2
4.895254396282715
-----
iteration: 3
3.456549842497898
-----
iteration: 4
2.671775952768653
-----
iteration: 5
1.7589610041687713
-----
iteration: 6
1.346433794749982
-----
iteration: 7
0.988562754829004
-----
iteration: 8
0.7226269091595829
-----
iteration: 9
0.5402742342279326
-----
iteration: 10
0.4240037520451201
-----
iteration: 11
0.33018757392135256
-----
iteration: 12
0.2667825582894579
-----
iteration: 13
0.20282385324479651
-----
iteration: 14
0.16500437484283018
-----
iteration: 15
0.14085570025759503
-----
iteration: 16
0.11800254199207774
-----
iteration: 17
0.10021288158430171
-----
iteration: 18
0.08516357960225346
-----
iteration: 19
0.06627078802932523
-----
iteration: 20
0.057373307978325634
-----
iteration: 21
0.04921144180138041
-----
iteration: 22
0.039492768443328614
-----
iteration: 23
0.03283060739625912
-----
iteration: 24
0.028230301856828913
-----
iteration: 25
0.022951439754857254
-----
iteration: 26
0.019658

In [37]:
async_convergence_threshold = 1*10**-5

sum_of_iterations = 0
num_iterations = 1


NODE_UPDT_PE = 84
PEs = 4

for it in range(0,num_iterations):
    # print(f"-------------- ITERATION = {it+1} --------------")
    iteration = 250
    while iteration > 250-1:
        P_i, mu_i, iteration = run_GaBP_HARDWARE_ACCELERATED_RESIDUAL_NEW(A, b, caching=True, node_updates_per_pe=NODE_UPDT_PE, number_pes=PEs, TRUE_MEAN=final_mean, max_iter=250, mae=True, convergence_threshold=async_convergence_threshold, show=True)
        break
    sum_of_iterations += iteration
    print(f"-------------- ITERATION = {it+1} => Streams = {iteration} --------------")

print(f"AVE. ASYNC ITERATIONS = {sum_of_iterations/num_iterations}")

iteration: 1
6.435668615220235
-----
iteration: 2
5.4395181521369365
-----
iteration: 3
4.289447950782878
-----
iteration: 4
4.274303652643519
-----
iteration: 5
4.263369469261251
-----
iteration: 6
4.228177532659867
-----
iteration: 7
4.228177532659867
-----
iteration: 8
4.153738434579054
-----
iteration: 9
3.8947938563330275
-----
iteration: 10
3.806975202680022
-----
iteration: 11
3.7770573754038415
-----
iteration: 12
3.718698076915047
-----
iteration: 13
3.5594449853570502
-----
iteration: 14
3.5474145756634714
-----
iteration: 15
3.3243788448251554
-----
iteration: 16
3.3107268923100905
-----
iteration: 17
3.3034176363897267
-----
iteration: 18
3.283488024971552
-----
iteration: 19
3.1180228219881654
-----
iteration: 20
3.0282842659748304
-----
iteration: 21
2.865860123455599
-----
iteration: 22
2.844754676423354
-----
iteration: 23
2.8133558628657704
-----
iteration: 24
2.7735257938982962
-----
iteration: 25
2.6126773912505072
-----
iteration: 26
2.449977977988976
-----
iteratio

In [20]:
async_convergence_threshold = 1*10**-5

sum_of_iterations = 0
num_iterations = 1


NODE_UPDT_PE = 84
PEs = 4

for it in range(0,num_iterations):
    # print(f"-------------- ITERATION = {it+1} --------------")
    iteration = 100
    while iteration > 100-1:
        P_i, mu_i, iteration, T_max_lst, PADDING_ANALYTICS = run_GaBP_HARDWARE_ACCELERATED_RESIDUAL_T_nP(A, b, caching=True, node_updates_per_pe=NODE_UPDT_PE, number_pes=PEs, TRUE_MEAN=final_mean, max_iter=100, mae=True, convergence_threshold=async_convergence_threshold, show=True)
        break
    sum_of_iterations += iteration
    print(f"-------------- ITERATION = {it+1} => Streams = {iteration} --------------")

print(f"AVE. ASYNC ITERATIONS = {sum_of_iterations/num_iterations}")

iteration: 1
1.5397747028261963
-----
iteration: 2
1.2362695098423855
-----
iteration: 3
0.9074741768119263
-----
iteration: 4
0.7514266892531033
-----
iteration: 5
0.6618460817407389
-----
iteration: 6
0.6528108925870889
-----
iteration: 7
0.611506034822715
-----
iteration: 8
0.5964456965501264
-----
iteration: 9
0.5957697112627308
-----
iteration: 10
0.5955145755577919
-----
iteration: 11
0.5954582648031882
-----
iteration: 12
0.5954351436688587
-----
iteration: 13
0.5954351435850983
-----
iteration: 14
0.5954351436121816
-----
iteration: 15
0.5954351435863012
-----
iteration: 16
0.5954351435703302
-----
iteration: 17
0.5954351437824752
-----
iteration: 18
0.5954351437395756
-----
iteration: 19
0.5954351435979041
-----
iteration: 20
0.5954351435839749
-----
iteration: 21
0.5954351434074984
-----
iteration: 22
0.5954351435528724
-----
iteration: 23
0.595435143598548
-----
iteration: 24
0.5954351438346293
-----
iteration: 25
0.5954351436749665
-----
iteration: 26
0.5954351435267077
---

In [17]:
PADDING_ANALYTICS

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]